https://dacon.io/codeshare/2288?dtype=recent&s_id=0 <br>
https://velog.io/@devseunggwan/Machine-Learning-pycaret%EC%9D%84-%EC%82%AC%EC%9A%A9%ED%95%9C-%EB%8D%B0%EC%9D%B4%ED%84%B0-%EB%B6%84%EC%84%9D

In [ ]:
!pip install pycaret

In [ ]:
# !pip install catboost

In [ ]:
# !pip install lightgbm

In [ ]:
from pycaret.regression import *
from google.colab import drive
from matplotlib import pyplot as plt
import pandas as pd
import numpy as np
# import optuna
from lightgbm import LGBMRegressor
# import catboost as cat
# from optuna import Trial
# from optuna.samplers import TPESampler
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split, cross_val_score, KFold

drive.mount('/content/gdrive')

train = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/train.csv")
test = pd.read_csv("/content/gdrive/My Drive/Colab Notebooks/test.csv")
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')
submission = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/sample_submission.csv')

# 데이터

## ① train set manipulation

In [ ]:
data = train

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').sort_values('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')

feature = pd.merge(feature, age, how = 'left', on = '지역')

feature['공가비율'] = feature['공가수'] / feature['총세대수']

feature['등록차량수'] = (feature['등록차량수'] - feature['단지내주차면수'])

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

train_target = np.array(feature['등록차량수'] / feature['단지내주차면수'])#.astype(int)


feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
train_feature = np.array(feature[feature_names_])

## ② test set manipulation

In [ ]:
data = test

data1 = data.copy().fillna(0)
feature = data1.drop_duplicates('단지코드').set_index('단지코드')
feature['주거세대수'] = feature['총세대수'] - feature['공가수']

data2 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data2['총전용면적'] = data2['전용면적'] * data2['전용면적별세대수']
data2 = data2.groupby('단지코드')['총전용면적'].sum().reset_index().set_index('단지코드')

data3 = data.copy()[['단지코드', '전용면적', '전용면적별세대수']].fillna(0)
data3['전용면적'] = np.round(data3['전용면적'], -1).astype(int)
list2 = []
for i in range(10, 610, 10):
  list2.append('면적' + str(i))
  data3['면적' + str(i)] = (data3['전용면적'] == i) * data3['전용면적별세대수']
data3 = data3.groupby('단지코드')[list2].sum().reset_index().set_index('단지코드')

feature = pd.merge(feature, data2, on = '단지코드')
feature = pd.merge(feature, data3, on = '단지코드')
feature['평균전용면적'] = feature['총전용면적'] / feature['총세대수']
list1 = []
for i in train['지역'].unique():
  list1.append(i + '여부')
  feature[i + '여부'] = (feature['지역'] == i) * 1

feature['아파트여부'] = (feature['임대건물구분'] == '아파트') * 1
age = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/age_gender_info.csv')

feature = pd.merge(feature, age, how = 'left', on = '지역')

age = ["10대미만"] + [str(i) + '대' for i in range(10, 110, 10)]
for i in age:
  feature[i] = feature[i + "(여자)"] + feature[i + "(남자)"]

feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
test_feature = np.array(feature[feature_names_])

# 모델

In [ ]:
random_state_ = 0
n_estimators_ = 100

feature_names_ = ['평균전용면적', '공가수', '40대', '50대']
s = []
for j in range(40):
  scores = []
  learning_rate_ = 0.01

  train_target = np.array(feature['등록차량수'])
  train_feature = np.array(feature['단지내주차면수'])
  train_X, test_X, train_y, test_y = train_test_split(train_feature, train_target, test_size = 0.1, shuffle = True, random_state=random_state_)
  b = test_X.copy()
  c = test_y

  for i in range(10):
    train_target = np.array(feature['등록차량수'] / feature['단지내주차면수'])
    train_feature = np.array(feature[feature_names_])
    train_X, test_X, train_y, test_y = train_test_split(train_feature, train_target, test_size = 0.1, shuffle = True, random_state=random_state_)
    rgr = LGBMRegressor(n_estimators=n_estimators_,
                        random_state=random_state_,
                        learning_rate=learning_rate_)
    
    rgr.fit(train_X, train_y)
    score = mean_absolute_error(test_y, rgr.predict(test_X))
    learning_rate_ += 0.01

    a = rgr.predict(test_X)
    sss = mean_absolute_error(c, b * a)
    scores.append(sss)
  print(min(scores))
  s.append(min(scores))
  random_state_ += 1
print("LGBM AUC: %0.4f (+/- %0.4f)" %(np.mean(s), np.std(s) * 2))

117.61441327353589
115.88207499400868
134.6971888856049
85.43352625711017
141.1792888570643
135.08513595371787
112.92357260796183
134.1268999745059
105.85091852208119
129.91295242780956
107.27406010492884
89.98886540342407
138.26086270939155
93.39397485146785
115.83498259865523
134.7643617927108
119.42764512893427
107.47079951302199
98.36880566495282
90.45669535041408
132.7289062925234
100.9515872412667
86.475914179886
93.16921876935784
106.49971345007131
124.61877768593618
104.19454127760572
100.29164333430388
132.02698998136944
111.76625373759241
109.17008182376573
127.98338516349145
117.57984545746002
121.73169829801398
133.01436006203977
88.41911105421637
105.98328868301802
107.53320483668696
120.19612723763545
120.33587374562923
LGBM AUC: 113.8154 (+/- 31.7752)


## PyCaret

https://today-1.tistory.com/17

In [ ]:
X_caret = pd.DataFrame(train_feature, columns = feature_names_)
y_caret = pd.DataFrame(train_target, columns = ['등록차량수'])

final_data = pd.concat([X_caret, y_caret], axis = 1)

test_caret = pd.DataFrame(test_feature, columns = feature_names_)

In [ ]:
final_data

,평균전용면적,공가수,40대,50대,등록차량수
0,42.175230,10.0,0.154967,0.157023,0.098174
1,39.490921,3.0,0.152278,0.148467,-0.392157
2,53.441049,16.0,0.152278,0.148467,-0.604211
3,45.952500,16.0,0.159212,0.147345,0.487936
4,41.475843,5.0,0.143650,0.155790,-0.139733
...,...,...,...,...,...
418,41.832591,14.0,0.161562,0.142203,-0.068571
419,43.768635,23.0,0.122722,0.151754,-0.162485
420,40.569630,9.0,0.143650,0.155790,-0.275000
421,40.247098,19.0,0.144691,0.159442,0.046099


In [ ]:
clf = setup(data = final_data, target = '등록차량수') 

,Description,Value
0,session_id,4047
1,Target,등록차량수
2,Original Data,"(423, 5)"
3,Missing Values,False
4,Numeric Features,4
5,Categorical Features,0
6,Ordinal Features,False
7,High Cardinality Features,False
8,High Cardinality Method,None
9,Transformed Train Set,"(296, 4)"


In [ ]:
# dt = create_model('dt')
# bagged_dt = ensemble_model(dt) # 배깅
# boosted_dt = ensemble_model(dt, method='Boosting')
# tuned_rf = tune_model('rf')

In [ ]:
# blender = blend_models(estimator_list= [boosted_dt, bagged_dt, tuned_rf], method='soft')

In [ ]:
best_2 = compare_models(sort = 'MAE', n_select = 2)

,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lr,Linear Regression,0.2008,0.0733,0.2667,0.1843,0.1773,2.5085,0.010
lar,Least Angle Regression,0.2008,0.0733,0.2667,0.1843,0.1773,2.5085,0.012
huber,Huber Regressor,0.2022,0.0732,0.2667,0.1830,0.1777,2.5997,0.024
lightgbm,Light Gradient Boosting Machine,0.2050,0.0768,0.2723,0.1507,0.1682,2.4478,0.023
br,Bayesian Ridge,0.2074,0.0770,0.2745,0.1308,0.1832,2.2248,0.013
ridge,Ridge Regression,0.2090,0.0767,0.2742,0.1320,0.1838,2.2147,0.010
ada,AdaBoost Regressor,0.2092,0.0805,0.2790,0.1108,0.1732,2.1373,0.063
gbr,Gradient Boosting Regressor,0.2100,0.0813,0.2804,0.1012,0.1681,2.7461,0.041
et,Extra Trees Regressor,0.2109,0.0866,0.2899,0.0288,0.1665,2.6964,0.347
rf,Random Forest Regressor,0.2120,0.0800,0.2773,0.1158,0.1672,2.4090,0.398


In [ ]:
blended = blend_models(estimator_list = best_2, fold = 5)
pred_holdout = predict_model(blended)
final_model = finalize_model(blended)

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.1913,0.0849,0.2914,0.2921,0.1957,1.0763
1,0.2040,0.0742,0.2725,0.3423,0.1911,1.4064
2,0.1944,0.0665,0.2578,0.1960,0.1640,5.5578
3,0.2180,0.0775,0.2785,0.0997,0.1785,1.6853
4,0.1940,0.0617,0.2484,0.1070,0.1577,2.8646
Mean,0.2003,0.0730,0.2697,0.2074,0.1774,2.5181
SD,0.0098,0.0082,0.0152,0.0972,0.0148,1.6351


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,Voting Regressor,0.2481,0.1063,0.326,0.1111,0.2125,1.8191


In [ ]:
lr = create_model('lr')

,MAE,MSE,RMSE,R2,RMSLE,MAPE
0,0.1704,0.0468,0.2163,0.3725,0.1626,1.0964
1,0.2144,0.1245,0.3529,0.2003,0.2255,1.0809
2,0.2094,0.0834,0.2888,0.3047,0.2058,1.2100
3,0.2021,0.0665,0.2580,0.2764,0.1741,1.6740
4,0.1699,0.0388,0.1970,0.2757,0.1470,9.9471
5,0.2248,0.0968,0.3111,0.0889,0.1863,0.9481
6,0.2204,0.0799,0.2826,0.0758,0.1731,2.1092
7,0.2085,0.0722,0.2687,0.0755,0.1789,1.1894
8,0.2120,0.0796,0.2821,-0.0705,0.1701,2.7191
9,0.1758,0.0440,0.2097,0.2433,0.1493,3.1109


In [ ]:
final_model = finalize_model(lr)
pred = predict_model(final_model, data = test_caret)

In [ ]:
test_caret

,평균전용면적,공가수,40대,50대
0,48.269164,14.0,0.161562,0.142203
1,46.846145,9.0,0.161562,0.142203
2,45.167690,18.0,0.122722,0.151754
3,43.471619,7.0,0.143650,0.155790
4,45.841735,11.0,0.161562,0.142203
...,...,...,...,...
145,36.259885,17.0,0.153078,0.157333
146,35.555889,35.0,0.154967,0.157023
147,28.884500,9.0,0.144203,0.159443
148,32.054015,38.0,0.144691,0.159442


In [ ]:
pred

,평균전용면적,공가수,40대,50대,Label
0,48.269164,14.0,0.161562,0.142203,0.081569
1,46.846145,9.0,0.161562,0.142203,0.100963
2,45.167690,18.0,0.122722,0.151754,-0.193350
3,43.471619,7.0,0.143650,0.155790,-0.004109
4,45.841735,11.0,0.161562,0.142203,0.078134
...,...,...,...,...,...
145,36.259885,17.0,0.153078,0.157333,-0.079830
146,35.555889,35.0,0.154967,0.157023,-0.194565
147,28.884500,9.0,0.144203,0.159443,-0.147744
148,32.054015,38.0,0.144691,0.159442,-0.305976


In [ ]:
tmp = feature['단지내주차면수']
submission['num'] = np.round(tmp * pred['Label'] + tmp)
submission.to_csv('/content/gdrive/My Drive/Colab Notebooks/submissionJunLinearReg.csv', index = False)